In [4]:
import random
import math

NumEval = 0    # Total number of evaluations


def main():
    # Create an instance of TSP
    p = createProblem()    # 'p': (numCities, locations, table)
    # Call the search algorithm
    solution, minimum = steepestAscent(p)
    # Show the problem and algorithm settings
    describeProblem(p)
    displaySetting()
    # Report results
    displayResult(solution, minimum)
    
def createProblem():
    ## Read in a TSP (# of cities, locatioins) from a file.
    ## Then, create a problem instance and return it.
    fileName = input("Enter the file name of a TSP: ")
    infile = open(fileName, 'r')
    # First line is number of cities
    numCities = int(infile.readline())
    locations = []
    line = infile.readline()  # The rest of the lines are locations
    while line != '':
        locations.append(eval(line)) # Make a tuple and append
        line = infile.readline()
    infile.close()
    table = calcDistanceTable(numCities, locations)
    return numCities, locations, table


def calcDistanceTable(numCities, locations): ###
    ###
    table = []     # 전체 거리 값을 담는 행렬을 정의
    line_table = []  # 하나의 row마다 거리 값을 담는 list를 정의
    for i in range(0,numCities): # 첫번째 row부터 마지막 row까지
        for j in range(0,numCities): # 각 row의 첫번째 entry 부터 마지막 entry 까지 반복하며
            value = math.sqrt(pow(locations[i][0]-locations[j][0],2) + pow(locations[i][1]-locations[j][1],2)) # 두 점사이의 거리 계산
            line_table.append(value) # 하나의 row마다 처리하는 list인 line_table에 거리 value를 apeend.
        table.append(line_table)  # 하나의 row가 가지고 있는 거리를 전체 행렬 table에 append해줌
        line_table = [] # 직전 계산하였던 row의 거리 값들을 비운다.
    ###
    return table # A symmetric matrix of pairwise distances


def steepestAscent(p):
    current = randomInit(p)   # 'current' is a list of city ids
    valueC = evaluate(current, p)
    while True:
        neighbors = mutants(current, p)
        (successor, valueS) = bestOf(neighbors, p)
        if valueS >= valueC:
            break
        else:
            current = successor
            valueC = valueS
    return current, valueC

def randomInit(p):   # Return a random initial tour
    n = p[0]
    init = list(range(n))
    random.shuffle(init)
    return init


def evaluate(current, p): ###
    ## Calculate the tour cost of 'current'
    ## 'p' is a Problem instance
    ## 'current' is a list of city ids
    global NumEval  # NumEval을 전역변수로 선언한다.
    NumEval += 1    # 반복할때마다 1씩 더해준다. 즉, 총 반복한 횟수
    cost = 0       # 비용 cost를 0으로 초기화 한다.
    table = p[2]   # 좌표마다 거리값이 계산되어 있는 p[2] 를 table 변수에 할당한다.
    for i in range(len(current) - 2): # 한번 반복할때마다 다음번째 변수까지 처리하므로 총 index - 2 만큼 반복하며
        cost = cost + table[current[i]][current[i+1]] # 좌표를 움직이며 이동한 거리의 값을 계산한다.
        
    return cost


def mutants(current, p): # Apply inversion
    n = p[0]
    neighbors = []
    count = 0
    triedPairs = []
    while count <= n:  # Pick two random loci for inversion
        i, j = sorted([random.randrange(n) for _ in range(2)])
        if i < j and [i, j] not in triedPairs:
            triedPairs.append([i, j])
            curCopy = inversion(current, i, j)
            count += 1
            neighbors.append(curCopy)
    return neighbors

def inversion(current, i, j):  ## Perform inversion
    curCopy = current[:]
    while i < j:
        curCopy[i], curCopy[j] = curCopy[j], curCopy[i]
        i += 1
        j -= 1
    return curCopy

def bestOf(neighbors, p): 
    ###
    bestValue = evaluate(neighbors[0],p)    # bestValue를 evaluate(neighbors[0],p) 즉, 첫번째값으로 초기화 한다.
    best = neighbors[0]                     # best를 neighbors의 첫번째 리스트로 초기화 한다
    for i in range(len(neighbors)): # neighbors 개수만큼 반복하며
        if evaluate(neighbors[i],p) < bestValue: # neighbors[i] 가 bestValue 보다 작은 경우, 즉 더 작은 값인 경우
            best = neighbors[i]                # best(next successor) 를 neighbors[i]로 바꿔주고
            bestValue = evaluate(neighbors[i],p) # bestValue에 cost값을 할당해준다.
    ###
    return best, bestValue

def describeProblem(p):
    print()
    n = p[0]
    print("Number of cities:", n)
    print("City locations:")
    locations = p[1]
    for i in range(n):
        print("{0:>12}".format(str(locations[i])), end = '')
        if i % 5 == 4:
            print()

def displaySetting():
    print()
    print("Search algorithm: Steepest-Ascent Hill Climbing")

def displayResult(solution, minimum):
    print()
    print("Best order of visits:")
    tenPerRow(solution)       # Print 10 cities per row
    print("Minimum tour cost: {0:,}".format(round(minimum)))
    print()
    print("Total number of evaluations: {0:,}".format(NumEval))

def tenPerRow(solution):
    for i in range(len(solution)):
        print("{0:>5}".format(solution[i]), end='')
        if i % 10 == 9:
            print()

main()


Enter the file name of a TSP: tsp100.txt

Number of cities: 100
City locations:
     (49, 3)    (74, 73)    (65, 36)    (39, 41)    (61, 99)
    (69, 44)    (88, 92)    (97, 28)    (53, 64)    (30, 77)
    (96, 62)    (61, 45)     (30, 3)    (66, 41)     (18, 9)
    (61, 64)    (28, 88)     (2, 72)    (80, 66)    (56, 38)
    (51, 16)     (18, 2)    (89, 18)    (67, 66)     (72, 6)
    (53, 32)    (29, 25)    (77, 69)    (89, 56)    (68, 88)
    (98, 53)    (36, 25)     (16, 0)    (20, 32)   (100, 10)
    (49, 49)    (85, 38)    (42, 52)     (3, 85)    (62, 77)
    (97, 87)    (75, 54)    (40, 19)    (32, 33)     (59, 1)
    (90, 43)    (62, 11)    (77, 14)    (88, 66)    (39, 32)
    (34, 69)    (12, 73)    (58, 88)    (34, 19)    (32, 45)
    (36, 36)    (84, 47)    (28, 18)    (23, 57)    (14, 52)
    (29, 38)     (0, 17)    (87, 96)    (61, 11)    (45, 56)
     (2, 60)    (97, 67)    (73, 70)    (49, 94)    (88, 55)
    (40, 55)    (23, 27)    (33, 68)    (70, 84)     (20, 0)
    (